##$E_7$

In this notebook we show that all minuscule Coxeter matroids for $E_7$ satisfy strong exchange.

In [109]:
from copy import deepcopy
from itertools import combinations

In [111]:
s = {1,2,3}
for a in combinations(s, 2):
    print(a)

(1, 2)
(1, 3)
(2, 3)


In [196]:
class GossetSpace:
    def __init__(self):
        self.vertices= set()
        for i in range(1,8):
            for j in range(i):
                self.vertices.add((1, (j,i)))
                self.vertices.add((-1,(j,i)))
        self.create_dist_dict()

    def create_dist_dict(self):
        self.dist_dict = {}
        for v in self.vertices:
            v_dist_dict = {i: set() for i in range(4)}
            for w in self.vertices:
                v_dist_dict[self.dist(v,w)].add(w)
            self.dist_dict[v] = v_dist_dict
            
    def dist(self, vert1, vert2):
        sign1 = vert1[0]
        set1 = {*vert1[1]}
        sign2 = vert2[0]
        set2 = {*vert2[1]}
        if sign1 == sign2:
            return len(set1 - set2)
        else:
            return 1 + len(set1 & set2)

    def _opposite(self, v):
        sign, pair = v
        return -sign, pair

    def opposite(self, V):
        if type(V) == set:
            return {self._opposite(v) for v in V}
        else:
            return self._opposite(V)

    def sphere(self, vertex, radius):
        return self.dist_dict[vertex][radius]


class CounterExampleFinder():
    def __init__(
        self,
        graph,
        initial_feasibles = None
    ):
        self.graph = graph
        if initial_feasibles is not None:
            self.feasibles = initial_feasibles
            self.unsafe_pairs = {pair for pair in combinations(initial_feasibles,2) if graph.dist(*pair)==2}
################# TODO: Make this dict and use size of intervals as heuristic
            self.check_pairs()
        else:
            self.feasibles = set()
            self.unsafe_pairs = set()

    def find(self):
        if not self.unsafe_pairs:
            return self.feasibles
        v1, v2= self.unsafe_pairs.pop()
        graph_interval = self.graph.sphere(v1, 1) & self.graph.sphere(v2, 1) 
        feasible_interval = graph_interval & self.feasibles
        possible_new_vertices = graph_interval - self.graph.opposite(self.feasibles) - self.feasibles
        possible_additions = {pair for pair in combinations(possible_new_vertices | feasible_interval ,2) 
                              if self.graph.dist(*pair)==2}
        if not possible_additions:
            return None
        for v, w in possible_additions:
            recursion_solver=self.copy()
            recursion_solver.add_to_feasibles(v)
            recursion_solver.add_to_feasibles(w)
            if (w,v) in recursion_solver.unsafe_pairs:
                recursion_solver.unsafe_pairs.remove((w,v))
            elif (v,w) in recursion_solver.unsafe_pairs:
                recursion_solver.unsafe_pairs.remove((v,w))
            recursion_solver.check_pairs()
            recursion_result = recursion_solver.find()
            if recursion_result is not None:
                return recursion_result
        return None
        
        
    def add_to_feasibles(self, vert):
        if vert not in self.feasibles:
            self.unsafe_pairs |= {(vert , v) for v in self.feasibles if self.graph.dist(vert,v)==2}
            self.feasibles.add(vert)
    
    def check_pairs(self):
        for v1, v2 in list(self.unsafe_pairs):
            interval = self.graph.sphere(v1, 1) & self.graph.sphere(v2, 1) & self.feasibles
            for pair in combinations(interval, 2):
                if self.graph.dist(*pair)==2:
                    self.unsafe_pairs.remove((v1,v2))
                    break

    def copy(self):
        return deepcopy(self)

    
            
        
        

In [197]:
gossi = GossetSpace()
finder = CounterExampleFinder(
    gossi,
    initial_feasibles = {
        (1,(0,1)),
        (-1,(0,1)),
        (1,(0,2)),
        (-1,(1,3)),
        (1,(2,3)),
        (-1,(4,5)),
        (-1,(6,7)),
        (-1,(3,4)),
        (1,(0,4)),
    }
)
counter_example=finder.find()
print(counter_example)

None
